# AST to L0 data lineage file

Converting an Abstract Syntax Tree in the JSON form into the L0 data lineage file.

[Repository](https://github.com/jurgenei/SqlXmlAst) where the JSON AST is generated.

## Case 1: simple insert

In [10]:
import numpy as np
import json

with open('./plsql_insert_1.json') as f:
    ast = json.load(f)

### Get target table and columns

In [86]:
tgt_raw = ast['body'][1]['sql_script'][0]['insert_statement'][2]['single_table_insert'][0]['insert_into_clause']

tgt_tbl = tgt_raw[2]['regular_id'][0]['value']

tgt_col_raw = tgt_raw[4]['paren_column_list'][1]['column_list']
tgt_col = []

for c in tgt_col_raw:
    if 'rule-path' in c:
        tgt_col.append(c['regular_id'][0]['value'])

### Get source table and columns

In [87]:
src_raw = ast['body'][1]['sql_script'][0]['insert_statement'][2]['single_table_insert'][2]

src_tbl = src_raw['query_block'][4]['from_clause'][2]['regular_id'][0]['value']

src_col_raw = src_raw['query_block'][2]['expressions']
src_col = []

for c in src_col_raw:
    if 'object-type' in c and c['object-type'] == 'column_name':
        src_col.append(c['regular_id'][0]['value'])

### Result lineage

In [88]:
print('Source table: ' + src_tbl)
print('Source columns: ' + str(src_col))
print('\n')
print('Target table: ' + tgt_tbl)
print('Target columns: ' + str(tgt_col))

Source table: customers
Source columns: ['customer_id', 'last_name', 'first_name']


Target table: contacts
Target columns: ['contact_id', 'last_name', 'first_name']


## Using a DFT method
A depth first traversal method

In [122]:
lineage_raw = []

def dfs(ast, info):
    if type(ast) == dict:
        if 'rule-path' in ast and 'regular_id' in ast:
            info.append(ast)
            print(ast)
            print('\n')
            return
        else:
            for key in ast.keys():
                if 'where_clause' == key:
                    return
                dfs(ast[key], info)
    elif type(ast) == list:
        for item in ast:
            dfs(item, info)
#     elif type(ast) == str:
#         return
    else:
        return
        
    return


dfs(ast['body'][1]['sql_script'][0], lineage_raw)

{'rule-path': 'general_table_ref dml_table_expression_clause tableview_name identifier id_expression', 'object-type': 'tableview_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'contacts'}]}


{'rule-path': 'column_name identifier id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'contact_id'}]}


{'rule-path': 'column_name identifier id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'last_name'}]}


{'rule-path': 'column_name identifier id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'first_name'}]}


{'rule-path': 'expression logical_expression multiset_expression relational_expression compound_expression concatenation model_expression unary_expression atom general_element general_element_part id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type'

In [124]:
src_tbl = ''
src_col = []

tgt_tbl = ''
tgt_col = []

for item in lineage_raw:
    if 'general_table_ref' in item['rule-path']:
        tgt_tbl = item['regular_id'][0]['value']
    elif 'logical_expression' not in item['rule-path'] and 'column_name' == item['object-type']:
        tgt_col.append(item['regular_id'][0]['value'])
    elif 'logical_expression' in item['rule-path'] and 'column_name' == item['object-type']:
        src_col.append(item['regular_id'][0]['value'])
    else:
        src_tbl = item['regular_id'][0]['value']
        
        
print('Source table: ' + src_tbl)
print('Source columns: ' + str(src_col))
print('\n')
print('Target table: ' + tgt_tbl)
print('Target columns: ' + str(tgt_col))

Source table: customers
Source columns: ['customer_id', 'last_name', 'first_name']


Target table: contacts
Target columns: ['contact_id', 'last_name', 'first_name']


## Case 2: insert with complicated conditions

In [152]:
with open('./plsql_insert_2.json') as f:
    ast = json.load(f)

In [153]:
lineage_raw = []
dfs(ast['body'][1]['sql_script'][0], lineage_raw)

{'rule-path': 'general_table_ref dml_table_expression_clause tableview_name identifier id_expression', 'object-type': 'tableview_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'tt_bnk_dependent_defaulting'}]}


{'rule-path': 'expression logical_expression multiset_expression relational_expression compound_expression concatenation model_expression unary_expression atom general_element general_element_part id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'dependent_on_value'}]}


{'rule-path': 'expression logical_expression multiset_expression relational_expression compound_expression concatenation model_expression unary_expression atom general_element general_element_part id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'vortex_reference'}]}


{'rule-path': 'table_ref_aux_internal dml_table_expression_clause tableview_name identifier id_exp

In [129]:
src_tbl = []
src_col = []

tgt_tbl = ''
tgt_col = []

for item in lineage_raw:
    if 'general_table_ref' in item['rule-path']:
        tgt_tbl = item['regular_id'][0]['value']
    elif 'logical_expression' not in item['rule-path'] and 'column_name' == item['object-type']:
        tgt_col.append(item['regular_id'][0]['value'])
    elif 'logical_expression' in item['rule-path'] and 'column_name' == item['object-type']:
        src_col.append(item['regular_id'][0]['value'])
    else:
        src_tbl.append(item['regular_id'][0]['value'])
        
        
print('Source table: ' + str(src_tbl))
print('Source columns: ' + str(src_col))
print('\n')
print('Target table: ' + tgt_tbl)
print('Target columns: ' + str(tgt_col))

Source table: ['current_dependent_defaulting', 'dd', 'current_field_related_activity', 'fa']
Source columns: ['dependent_on_value', 'vortex_reference']


Target table: tt_bnk_dependent_defaulting
Target columns: []


The DFT returns some results, but there are two errors:
- Table alias not identified: we can easily solve it by ingnoring it
- Target columns not identified: this is because they are not defined explicitly. In this case, they are the same as source columns. We just have to assign the value to them.

In [154]:
src_tbl = []
src_col = []

tgt_tbl = ''
tgt_col = []

for item in lineage_raw:
    if 'table_alias' in item['rule-path']:
        continue
    if 'general_table_ref' in item['rule-path']:
        tgt_tbl = item['regular_id'][0]['value']
    elif 'logical_expression' not in item['rule-path'] and 'column_name' == item['object-type']:
        tgt_col.append(item['regular_id'][0]['value'])
    elif 'logical_expression' in item['rule-path'] and 'column_name' == item['object-type']:
        src_col.append(item['regular_id'][0]['value'])
    else:
        src_tbl.append(item['regular_id'][0]['value'])
        
        
print('Source table: ' + str(src_tbl))
print('Source columns: ' + str(src_col))
print('\n')
print('Target table: ' + tgt_tbl)
print('Target columns: ' + str(tgt_col))

Source table: ['current_dependent_defaulting', 'current_field_related_activity']
Source columns: ['dependent_on_value', 'vortex_reference']


Target table: tt_bnk_dependent_defaulting
Target columns: []


## Case 3, alias columns 1

In [155]:
import json

with open('./plsql_alias_column1.json') as f:
    ast = json.load(f)

In [156]:
lineage_raw = []
dfs(ast['body'][1]['sql_script'][0], lineage_raw)

src_tbl = []
src_col = []

tgt_tbl = ''
tgt_col = []

for item in lineage_raw:
    if 'table_alias' in item['rule-path'] or 'column_alias' in item['rule-path']:
        continue
    if 'general_table_ref' in item['rule-path']:
        tgt_tbl = item['regular_id'][0]['value']
    elif 'logical_expression' not in item['rule-path'] and 'column_name' == item['object-type']:
        tgt_col.append(item['regular_id'][0]['value'])
    elif 'logical_expression' in item['rule-path'] and 'column_name' == item['object-type']:
        src_col.append(item['regular_id'][0]['value'])
    else:
        src_tbl.append(item['regular_id'][0]['value'])
        
# when either target columns or table is identified, lease them as it is
if len(tgt_col) < 1 and tgt_tbl:
    tgt_col = src_col
        
        
print('Source table: ' + str(src_tbl))
print('Source columns: ' + str(src_col))
print('\n')
print('Target table: ' + tgt_tbl)
print('Target columns: ' + str(tgt_col))

{'rule-path': 'select_list_elements expressions expression logical_expression multiset_expression relational_expression compound_expression concatenation model_expression unary_expression atom general_element general_element_part id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'first_name'}]}


{'rule-path': 'identifier id_expression', 'object-type': 'column_alias', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'forename'}]}


{'rule-path': 'select_list_elements expressions expression logical_expression multiset_expression relational_expression compound_expression concatenation model_expression unary_expression atom general_element general_element_part id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'last_name'}]}


{'rule-path': 'identifier id_expression', 'object-type': 'column_alias', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 

The source table is wrong. This is because the if-else is not well defined. It is alright to keep adding if-else, but we will think more of a template way, which is the target for next step.

## Case 4, alias columns 2

In [151]:
import json

with open('./plsql_alias_column2.json') as f:
    ast = json.load(f)
    
lineage_raw = []
dfs(ast['body'][1]['sql_script'][0], lineage_raw)

src_tbl = []
src_col = []

tgt_tbl = ''
tgt_col = []

for item in lineage_raw:
    if 'table_alias' in item['rule-path'] or 'column_alias' in item['rule-path']:
        continue
    if 'general_table_ref' in item['rule-path']:
        tgt_tbl = item['regular_id'][0]['value']
    elif 'logical_expression' not in item['rule-path'] and 'column_name' == item['object-type']:
        tgt_col.append(item['regular_id'][0]['value'])
    elif 'logical_expression' in item['rule-path'] and 'column_name' == item['object-type']:
        src_col.append(item['regular_id'][0]['value'])
    else:
        src_tbl.append(item['regular_id'][0]['value'])
        
# when either target columns or table is identified, leave them as it is
if len(tgt_col) < 1 and tgt_tbl:
    tgt_col = src_col
        
        
print('Source table: ' + str(src_tbl))
print('Source columns: ' + str(src_col))
print('\n')
print('Target table: ' + tgt_tbl)
print('Target columns: ' + str(tgt_col))

{'rule-path': 'concatenation model_expression unary_expression atom general_element general_element_part id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'first_name'}]}


{'rule-path': 'concatenation model_expression unary_expression atom general_element general_element_part id_expression', 'object-type': 'column_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'last_name'}]}


{'rule-path': 'table_ref_list table_ref table_ref_aux table_ref_aux_internal dml_table_expression_clause tableview_name identifier id_expression', 'object-type': 'tableview_name', 'regular_id': [{'class': 'token', 'type': 'REGULAR_ID', 'value': 'employees'}]}


Source table: ['employees']
Source columns: []


Target table: 
Target columns: ['first_name', 'last_name']


Now we can see the shortage of extracting lineage info out of the DFT result: too many if-elses!

Possible improvement:
- DFT is ordered. So everything inside 'select_clause' should be considered in source info